<a href="https://colab.research.google.com/github/s-choung/CCEL_Multiscale/blob/main/carbide_data_generation_1205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OCP installation

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
# Set the path to your pseudopotential files in Google Drive
pp_drive_path = '/content/drive/My Drive/potcars'  # Update this path to your actual directory
# Set the VASP_PP_PATH environment variable
os.environ['VASP_PP_PATH'] = pp_drive_path


Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .
!pip -q install  lmdb pymatgen==2023.5.10 ase==3.22.1 orjson e3nn==0.2.6 wandb
!pip install submitit

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Cloning into 'ocp'...
remote: Enumerating objects: 11841, done.
remote: Counting objects: 100% (2729/2729), done.
remote: Compressing objects: 100% (730/730), done.
remote: Total 11841 (delta 2314), reused 2136 (delta 1999), pack-reused 9112
Receiving objects: 100% (11841/11841), 172.66 MiB | 31.98 MiB/s, done.
Resolving deltas: 100% (8167/8167), done.
Obtaining file:///content/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable f

In [ ]:
import json,zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm

from ase.io import read

from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from ocpmodels.datasets import SinglePointLmdbDataset
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import CGCNN,SchNet
from ocpmodels.datasets import data_list_collater
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
torch.cuda.is_available()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


False

In [ ]:
!sudo apt-get install povray  ## visulization program

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-povray libsdl1.2debian povray-includes
Suggested packages:
  povray-doc povray-examples
The following NEW packages will be installed:
  fonts-povray libsdl1.2debian povray povray-includes
0 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,792 kB of archives.
After this operation, 5,199 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-povray all 1:3.7.0.10-1 [70.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libsdl1.2debian amd64 1.2.15+dfsg2-6 [177 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 povray amd64 1:3.7.0.10-1 [1,312 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 povray-includes all 1:3.7.0.10-1 [233 kB]
Fetched 1,792 kB in 0s (4,256 kB/s)
debconf: unable to initialize frontend: Dialo

In [ ]:
from ase import Atoms
from ase.io import Trajectory, write
from ase.build import bulk, surface, molecule, add_adsorbate, fcc111
from ase.constraints import ExpCellFilter, StrainFilter, FixAtoms, FixedPlane, FixBondLength
from ase.optimize import LBFGS, BFGS, FIRE
from ase.neb import NEB
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.visualize import view
from ase.build.rotate import minimize_rotation_and_translation
from ase.md import MDLogger
from ase.io.vasp import read_vasp

import pandas as pd
import ipywidgets as widgets
from IPython.display import display_png, Image as ImageWidget
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import shutil

import glob
from pathlib import Path
from PIL import Image, ImageDraw

from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.vasp import Poscar
from pymatgen.core.surface import SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor

import numpy as np
from scipy.spatial.distance import cdist
def visual(structure, index, max_size=(100, 100), stretch_y=1.0):
    renderer = write('./temp.pov', structure, rotation='15z,-90x')
    renderer.render()
    image_path = './temp.png'
    img = Image.open(image_path)

    # Calculate new size with stretch factor
    new_size = (max_size[0], int(max_size[1] * stretch_y))
    img = img.resize(new_size, Image.LANCZOS)
    display(img)
    # Move files to output directory
    files = ['./temp.ini', './temp.pov', './temp.png']
    destination = './output/'

    # Ensure destination directory exists
    os.makedirs(destination, exist_ok=True)

    for file in files:
        # Remove the file in the destination directory if it exists
        if os.path.isfile(os.path.join(destination, os.path.basename(file))):
            os.remove(os.path.join(destination, os.path.basename(file)))

        shutil.move(file, destination)


## load some useful utillities
 code taken from https://open-catalyst-project.github.io/tutorial/fine-tuning/fine-tuning-oxides.html

In [ ]:
import ocpmodels as om
from pathlib import Path
import urllib
import os
from pathlib import Path
import requests
import subprocess
import sys
import numba
import numpy as np
import ase
import e3nn
import pymatgen.core as pc
import torch
import torch.cuda as tc
import torch_geometric as tg
import platform
import psutil

def ocp_root():
    """Return the root directory of the installed ocp package."""
    return Path(om.__file__).parent.parent

def ocp_main():
    """Return the path to ocp main.py"""
    return ocp_root() / "main.py"
def describe_ocp():
    """Print some system information that could be useful in debugging."""
    print(sys.executable, sys.version)
    print(f'ocp is installed at {ocp_root()}')

    commit_hash = (
            subprocess.check_output(
                    [
                        "git",
                        "-C",
                        om.__path__[0],
                        "describe",
                        "--always",
                    ]
                )
                .strip()
                .decode("ascii")
            )
    print(f'ocp repo is at git commit: {commit_hash}')
    print(f'numba: {numba.__version__}')
    print(f'numpy: {np.version.version}')
    print(f'ase: {ase.__version__}')
    print(f'e3nn: {e3nn.__version__}')
    print(f'pymatgen: {pc.__version__}')
    print(f'torch: {torch.version.__version__}')
    print(f'torch.version.cuda: {torch.version.cuda}')
    print(f'torch.cuda: is_available: {tc.is_available()}')
    if tc.is_available():
        print('  __CUDNN VERSION:', torch.backends.cudnn.version())
        print('  __Number CUDA Devices:', torch.cuda.device_count())
        print('  __CUDA Device Name:',torch.cuda.get_device_name(0))
        print('  __CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
    print(f'torch geometric: {tg.__version__}')
    print()
    print(f'Platform: {platform.platform()}')
    print(f'  Processor: {platform.processor()}')
    print(f'  Virtual memory: {psutil.virtual_memory()}')
    print(f'  Swap memory: {psutil.swap_memory()}')
    print(f'  Disk usage: {psutil.disk_usage("/")}')


checkpoints = {
    # Open Catalyst 2020 (OC20)
    'CGCNN 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_200k.pt',
    'CGCNN 2M'	    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_2M.pt',
    'CGCNN 20M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_20M.pt',
    'CGCNN All'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_all.pt',
    'DimeNet 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/dimenet_200k.pt',
    'DimeNet 2M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/dimenet_2M.pt',
    'SchNet 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_200k.pt',
    'SchNet 2M'	    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_2M.pt',
    'SchNet 20M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_20M.pt',
    'SchNet All'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_all_large.pt',
    'DimeNet++ 200k'   :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_200k.pt',
    'DimeNet++ 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_2M.pt',
    'DimeNet++ 20M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_20M.pt',
    'DimeNet++ All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_all.pt',
    'SpinConv 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_12/s2ef/spinconv_force_centric_2M.pt',
    'SpinConv All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/spinconv_force_centric_all.pt',
    'GemNet-dT 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_12/s2ef/gemnet_t_direct_h512_2M.pt',
    'GemNet-dT All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt',
    'PaiNN All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_05/s2ef/painn_h512_s2ef_all.pt',
    'GemNet-OC 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_base_s2ef_2M.pt',
    'GemNet-OC All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_base_s2ef_all.pt',
    'GemNet-OC All+MD'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/gemnet_oc_base_s2ef_all_md.pt',
    'GemNet-OC-Large All+MD' :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_large_s2ef_all_md.pt',
    'SCN 2M'   :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_t1_b1_s2ef_2M.pt',
    'SCN-t4-b2 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_t4_b2_s2ef_2M.pt',
    'SCN All+MD' :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_all_md_s2ef.pt',
    'eSCN-L4-M2-Lay12 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l4_m2_lay12_2M_s2ef.pt',
    'eSCN-L6-M2-Lay12 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m2_lay12_2M_s2ef.pt',
    'eSCN-L6-M2-Lay12 All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m2_lay12_all_md_s2ef.pt',
    'eSCN-L6-M3-Lay20 All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m3_lay20_all_md_s2ef.pt',
    'EquiformerV2 (83M) 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_83M_2M.pt',
    'EquiformerV2 (31M) All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_31M_ec4_allmd.pt',
    'EquiformerV2 (153M) All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_153M_ec4_allmd.pt',
    # Open Catalyst 2022 (OC22)
    'GemNet-dT OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gndt_oc22_all_s2ef.pt',
    'GemNet-OC OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_all_s2ef.pt',
    'GemNet-OC OC20+OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt',
    'GemNet-OC trained with `enforce_max_neighbors_strictly=False` #467 OC20+OC22' : 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_05/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt',
    'GemNet-OC OC20->OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_finetune_all_s2ef.pt',
    'equiformer' : 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_10/oc22/s2ef/eq2_121M_e4_f100_oc22_s2ef.pt'
                }

def list_checkpoints():
    """List checkpoints that are available to download."""
    print('See https://github.com/Open-Catalyst-Project/ocp/blob/main/MODELS.md for more details.')
    for key in checkpoints:
        print(key)
    print('Copy one of these keys to get_checkpoint(key) to download it.')


def get_checkpoint(key):
    """Download a checkpoint.

    key: string in checkpoints.

    Returns name of checkpoint that was saved.
    """
    url = checkpoints.get(key, None)
    if url is None:
        raise Exception('No url found for {key}')

    pt = Path(urllib.parse.urlparse(url).path).name

    if not os.path.exists(pt):
        with open(pt, 'wb') as f:
            print(f'Downloading {url}')
            f.write(requests.get(url).content)
    return pt

In [ ]:
# @title
from pathlib import Path
import numpy as np
from ase.db import connect

def train_test_val_split(ase_db, ttv=(0.8, 0.1, .1), files=('train.db', 'test.db', 'val.db'), seed=42):
    """Split an ase db into train, test and validation dbs.

    ase_db: path to an ase db containing all the data.
    ttv: a tuple containing the fraction of train, test and val data. This will be normalized.
    files: a tuple of filenames to write the splits into. An exception is raised if these exist.
           You should delete them first.
    seed: an integer for the random number generator seed

    Returns the absolute path to files.
    """

    for db in files:
        if os.path.exists(db):
            raise Exception('{db} exists. Please delete it before proceeding.')

    src = connect(ase_db)
    N = src.count()

    ttv = np.array(ttv)
    ttv /= ttv.sum()

    train_end = int(N * ttv[0])
    test_end = train_end + int(N * ttv[1])

    train = connect(files[0])
    test = connect(files[1])
    val = connect(files[2])

    ids = np.arange(1, N + 1)
    rng = np.random.default_rng(seed=42)
    rng.shuffle(ids)

    for _id in ids[0:train_end]:
        row = src.get(id=int(_id))
        train.write(row.toatoms())

    for _id in ids[train_end:test_end]:
        row = src.get(id=int(_id))
        test.write(row.toatoms())

    for _id in ids[test_end:]:
        row = src.get(id=int(_id))
        val.write(row.toatoms())

    return [Path(f).absolute() for f in files]

# @title
from yaml import load, dump
from yaml import CLoader as Loader, CDumper as Dumper
import torch
import os
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
from io import StringIO
import sys
import contextlib

def generate_yml_config(checkpoint_path, yml='run.yml', delete=(), update=()):
    """Generate a yml config file from an existing checkpoint file.

    checkpoint_path: string to path of an existing checkpoint
    yml: name of file to write to.
    pop: list of keys to remove from the config
    update: dictionary of key:values to update

    Use a dot notation in update.

    Returns an absolute path to the generated yml file.
    """

    # You can't just read in the checkpoint with torch. The calculator does some things to it.
    # Rather than recreate that here I just reuse the calculator machinery. I don't want to
    # see the output though, so I capture it.

    with contextlib.redirect_stdout(StringIO()) as _:
        config = OCPCalculator(checkpoint_path=checkpoint_path).config # there seems an parameter checkpoint changed in OCPcalculator (s.choung)

    for key in delete:
        if key in config and len(key.split('.')) == 1:
            del config[key]
        else:
            keys = key.split('.')
            if keys[0] in config:
                d = config[keys[0]]
            else:
                continue
            if isinstance(d, dict):
                for k in keys[1:]:
                    if isinstance(d[k], dict):
                        d = d[k]
                    else:
                        if k in d:
                            del d[k]

    def nested_set(dic, keys, value):
        for key in keys[:-1]:
            dic = dic.setdefault(key, {})
        dic[keys[-1]] = value

    for _key in update:
        keys = _key.split('.')
        nested_set(config, keys, update[_key])


    out = dump(config)
    with open(yml, 'wb') as f:
        f.write(out.encode('utf-8'))

    return Path(yml).absolute()

# OCDATA module installation

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
if not os.path.exists('tutorial'):
     !git clone https://github.com/Open-Catalyst-Project/tutorial.git
!git clone https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset.git

!pip install -e ./Open-Catalyst-Dataset
os.chdir('Open-Catalyst-Dataset')
!git clone -b OC22_dataset https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset.git


Cloning into 'tutorial'...
remote: Enumerating objects: 2147, done.
remote: Counting objects: 100% (1408/1408), done.
remote: Compressing objects: 100% (1134/1134), done.
remote: Total 2147 (delta 311), reused 1328 (delta 249), pack-reused 739
Receiving objects: 100% (2147/2147), 29.67 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (560/560), done.
Cloning into 'Open-Catalyst-Dataset'...
remote: Enumerating objects: 1822, done.
remote: Counting objects: 100% (514/514), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 1822 (delta 315), reused 340 (delta 284), pack-reused 1308
Receiving objects: 100% (1822/1822), 36.89 MiB | 12.78 MiB/s, done.
Resolving deltas: 100% (1137/1137), done.
Obtaining file:///content/ocp/Open-Catalyst-Dataset
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ocdata
Cloning into 'Open-Catalyst-Dataset'...
remote: Enumerating objects: 1822, done.
remote: Counting objects: 100% (514/514), done.
remote: Compressing objec

example to use Bulk module.

'/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'
이 피클파일에는 아래 기준을 통과한 안정한 bulk구조들만 담겨있음.


Pymatgen (63) was used to search over all bulk materials in the Materials Project with nonpositive formation energies and energies above lower hulls of at most 0.1 eV/atom.
https://pubs.acs.org/doi/full/10.1021/acscatal.0c04525

In [ ]:
import os
from ocdata.core.bulk import Bulk

# Path to the bulk database pickle file
bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'

# Initialize a Bulk object in various ways

# 1. Initialize a Bulk object by randomly selecting from the bulk database
bulk_random = Bulk(bulk_db_path=bulk_db_path)
print("Random Bulk:", bulk_random)

# 2. Initialize a Bulk object with a specific bulk ID from the database
bulk_id = 5  # Example bulk ID
bulk_with_id = Bulk(bulk_id_from_db=bulk_id, bulk_db_path=bulk_db_path)
print("Bulk with ID:", bulk_with_id)

# 3. Initialize a Bulk object with a specific source ID from the database
bulk_src_id = "mp-30"  # Example source ID
bulk_with_src_id = Bulk(bulk_src_id_from_db=bulk_src_id, bulk_db_path=bulk_db_path)
print("Bulk with Source ID:", bulk_with_src_id)

# Generate slabs from a Bulk object
max_miller_index = 1
slabs = bulk_random.get_slabs(max_miller=max_miller_index)
print(f"Number of slabs generated from random bulk: {len(slabs)}")

# Each slab can be further processed or analyzed as required
for i, slab in enumerate(slabs):
    print(f"Slab {i}: {slab}")


Random Bulk: Bulk: (Ca4Cd4Pd4)
Bulk with ID: Bulk: (In)
Bulk with Source ID: Bulk: (Cu)
Number of slabs generated from random bulk: 44
Slab 0: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.0, True)
Slab 1: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.025930058333333283, True)
Slab 2: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.07601833833333338, True)
Slab 3: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.12156635666666682, True)
Slab 4: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.1416974500000001, True)
Slab 5: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.16666666666666674, True)
Slab 6: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.025930058333333283, False)
Slab 7: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.07601833833333338, False)
Slab 8: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.12156635666666682, False)
Slab 9: Slab: (Ca12Cd12Pd12, (1, 1, 1), 0.1416974500000001, False)
Slab 10: Slab: (Ca8Cd8Pd8, (1, 1, 0), 0.02814837749999996, True)
Slab 11: Slab: (Ca8Cd8Pd8, (1, 1, 0), 0.07956871749999994, True)
Slab 12: Slab: (Ca8Cd8Pd8, (1, 1, 0), 0.12499999999999992, True)
Slab 13: Sla

In [ ]:
import pickle
from ocdata.core.bulk import Bulk

bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'

# Load the entire bulk database
with open(bulk_db_path, 'rb') as file:
    bulk_db = pickle.load(file)

all_bulks = []
all_bulk_atoms_list=[]
for bulk_entry in bulk_db:
    # Using the atoms directly
    bulk_atoms = bulk_entry['atoms']
    bulk_obj = Bulk(bulk_atoms=bulk_atoms)
    all_bulks.append(bulk_obj)
    all_bulk_atoms_list.append(bulk_atoms)

print(len(all_bulk_atoms_list))
print(all_bulk_atoms_list[0].symbols)


11410
Re2


carbide선별


In [ ]:
# Initialize an empty list to store bulks containing carbon
carbon_containing_bulks = []

# Iterate over all bulk structures
for bulk_atoms in all_bulk_atoms_list:
    # Check if 'C' is in the symbols of the bulk structure
    if 'C' in bulk_atoms.symbols:
        carbon_containing_bulks.append(bulk_atoms)

# Now, carbon_containing_bulks contains only those bulks which have carbon
print(f"Number of bulks containing carbon: {len(carbon_containing_bulks)}")

# Optionally, print out the symbols for the first few carbon-containing bulks to verify
for i, bulk_atoms in enumerate(carbon_containing_bulks):  # Adjust the range as needed
    print(f"Bulk {i} symbols: {bulk_atoms.symbols}")


Number of bulks containing carbon: 147
Bulk 0 symbols: C4
Bulk 1 symbols: Si19C19
Bulk 2 symbols: C7Si7
Bulk 3 symbols: C12Mn28
Bulk 4 symbols: Nb8C4
Bulk 5 symbols: CNb2
Bulk 6 symbols: Si8C8
Bulk 7 symbols: C3Al4
Bulk 8 symbols: Ti6C4
Bulk 9 symbols: C4Mn10
Bulk 10 symbols: CTi
Bulk 11 symbols: Cr6C4
Bulk 12 symbols: Si8C8
Bulk 13 symbols: C4W8
Bulk 14 symbols: W18C8
Bulk 15 symbols: C5Si5
Bulk 16 symbols: H4C
Bulk 17 symbols: Re4C2
Bulk 18 symbols: CSi
Bulk 19 symbols: W6C3
Bulk 20 symbols: C4V8
Bulk 21 symbols: NbC
Bulk 22 symbols: CS14
Bulk 23 symbols: CrC
Bulk 24 symbols: Si10C10
Bulk 25 symbols: V2C
Bulk 26 symbols: CMo
Bulk 27 symbols: Ti8C5
Bulk 28 symbols: Si21C21
Bulk 29 symbols: C12Cr28
Bulk 30 symbols: C2Si2
Bulk 31 symbols: Si14C14
Bulk 32 symbols: W4C2
Bulk 33 symbols: H2C2
Bulk 34 symbols: C8Cr12
Bulk 35 symbols: C5Nb6
Bulk 36 symbols: Si5C5
Bulk 37 symbols: C4Ti8
Bulk 38 symbols: C4Mo8
Bulk 39 symbols: C6Si6
Bulk 40 symbols: C4Si4
Bulk 41 symbols: K2C16
Bulk 42 symbols


아래처럼 원하는 elements to do형태로 짜서 cabide중에서 H4C같은애들 걸러서 쓰면될듯

```
from ase import Atoms
from pymatgen.core.periodic_table import Element
import pickle
from ocdata.core.bulk import Bulk

# Define the screening criteria for elements
elements_to_do = [el.symbol for el in Element if el.number < 80 and 21 < el.number and 3 < el.group < 16]
exclude_elements = ['Pb', 'Tl', 'Hg', 'Cd', 'Mg','As']  # Elements to exclude
elements_to_do = [el for el in elements_to_do if el not in exclude_elements]
print(elements_to_do)
# Load the entire bulk database
bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'
with open(bulk_db_path, 'rb') as file:
    bulk_db = pickle.load(file)

# Initialize lists for unary and binary bulks
unary_bulks = []
binary_bulks = []

# Iterate over the bulk database
for bulk_entry in bulk_db:
    bulk_atoms = bulk_entry['atoms']
    unique_elements = set(bulk_atoms.get_chemical_symbols())

    # Check if the bulk is unary or binary and matches the criteria
    if len(unique_elements) in [1, 2] and all(el in elements_to_do for el in unique_elements):
        if len(unique_elements) == 1:
            unary_bulks.append(bulk_atoms)
        else:
            binary_bulks.append(bulk_atoms)

# Print the results
print(f"Number of unary bulks: {len(unary_bulks)}")
print(f"Number of binary bulks: {len(binary_bulks)}")
print(unary_bulks)
print(binary_bulks)```



일단 Multi carbide는 버리고 Unary carbide에 대해서만 해봅시다.

In [ ]:
from collections import Counter

# Initialize lists to store two-element and multi-element bulks
two_element_bulks = []
multi_element_bulks = []

# Iterate over the carbon-containing bulks
for bulk_atoms in carbon_containing_bulks:
    elements = bulk_atoms.symbols
    element_count = Counter(elements)

    # Check the number of unique elements
    if len(element_count) == 2:
        two_element_bulks.append(bulk_atoms)
    else:
        multi_element_bulks.append(bulk_atoms)

# Displaying the results
print(f"Number of two-element bulks: {len(two_element_bulks)}")
print(f"Number of multi-element bulks: {len(multi_element_bulks)}")

# Optionally, print out symbols for a few examples from each category
print("Two-element bulks examples:")
for i, bulk_atoms in enumerate(two_element_bulks):  # Adjust the range as needed
    print(f"Bulk {i} symbols: {bulk_atoms.symbols}")

print("\nMulti-element bulks examples:")
for i, bulk_atoms in enumerate(multi_element_bulks):  # Adjust the range as needed
    print(f"Bulk {i} symbols: {bulk_atoms.symbols}")
print("two_element_ carbide",len(two_element_bulks))
print("multi_element_ carbide",len(multi_element_bulks))


Number of two-element bulks: 51
Number of multi-element bulks: 96
Two-element bulks examples:
Bulk 0 symbols: Si19C19
Bulk 1 symbols: C7Si7
Bulk 2 symbols: C12Mn28
Bulk 3 symbols: Nb8C4
Bulk 4 symbols: CNb2
Bulk 5 symbols: Si8C8
Bulk 6 symbols: C3Al4
Bulk 7 symbols: Ti6C4
Bulk 8 symbols: C4Mn10
Bulk 9 symbols: CTi
Bulk 10 symbols: Cr6C4
Bulk 11 symbols: Si8C8
Bulk 12 symbols: C4W8
Bulk 13 symbols: W18C8
Bulk 14 symbols: C5Si5
Bulk 15 symbols: H4C
Bulk 16 symbols: Re4C2
Bulk 17 symbols: CSi
Bulk 18 symbols: W6C3
Bulk 19 symbols: C4V8
Bulk 20 symbols: NbC
Bulk 21 symbols: CS14
Bulk 22 symbols: CrC
Bulk 23 symbols: Si10C10
Bulk 24 symbols: V2C
Bulk 25 symbols: CMo
Bulk 26 symbols: Ti8C5
Bulk 27 symbols: Si21C21
Bulk 28 symbols: C12Cr28
Bulk 29 symbols: C2Si2
Bulk 30 symbols: Si14C14
Bulk 31 symbols: W4C2
Bulk 32 symbols: H2C2
Bulk 33 symbols: C8Cr12
Bulk 34 symbols: C5Nb6
Bulk 35 symbols: Si5C5
Bulk 36 symbols: C4Ti8
Bulk 37 symbols: C4Mo8
Bulk 38 symbols: C6Si6
Bulk 39 symbols: C4Si4
Bul

In [ ]:
from ocdata.core.bulk import Bulk
from ocdata.core.bulk import Slab
# Load the entire bulk database
bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'
with open(bulk_db_path, 'rb') as file:
    bulk_db = pickle.load(file)

# Get the second bulk ID and source ID
second_bulk_id, second_bulk_src_id = list(two_element_bulk_ids.items())[2]

# Initialize the Bulk object using the second bulk_id or bulk_src_id
if second_bulk_src_id is not None:
    bulk_obj = Bulk(bulk_src_id_from_db=second_bulk_src_id, bulk_db_path=bulk_db_path)
else:
    bulk_obj = Bulk(bulk_id_from_db=second_bulk_id, bulk_db_path=bulk_db_path)

# Define specific Miller indices you want to generate slabs for
specific_millers = (1, 1, 1)  # Example Miller indices

# Generate slabs for the specific Miller indices
second_bulk_slabs = Slab.from_bulk_get_all_slabs(bulk=bulk_obj,max_miller=1)

# Print the number of slabs generated and their details
print(f"Number of slabs generated from second bulk ID {second_bulk_id} for Miller indices {specific_millers}: {len(second_bulk_slabs)}")
for slab in second_bulk_slabs:
    print(slab)


Number of slabs generated from second bulk ID 1594 for Miller indices (1, 1, 1): 71
Slab: (C24Mn56, (1, 1, 1), 0.04001861250000005, True)
Slab: (C24Mn56, (1, 1, 1), 0.06669328249999995, True)
Slab: (C24Mn56, (1, 1, 1), 0.11050325999999999, True)
Slab: (C24Mn56, (1, 1, 1), 0.17533166500000008, True)
Slab: (C24Mn56, (1, 1, 1), 0.25, True)
Slab: (C24Mn56, (1, 1, 1), 0.49553619500000023, True)
Slab: (C24Mn56, (1, 1, 1), 0.04001861250000005, False)
Slab: (C24Mn56, (1, 1, 1), 0.06669328249999995, False)
Slab: (C24Mn56, (1, 1, 1), 0.11050325999999999, False)
Slab: (C24Mn56, (1, 1, 1), 0.17533166500000008, False)
Slab: (C24Mn56, (1, 1, 0), 0.014496740000000008, True)
Slab: (C24Mn56, (1, 1, 0), 0.04730328499999997, True)
Slab: (C24Mn56, (1, 1, 0), 0.07589165500000003, True)
Slab: (C24Mn56, (1, 1, 0), 0.09961352000000004, True)
Slab: (C24Mn56, (1, 1, 0), 0.125, True)
Slab: (C24Mn56, (1, 1, 0), 0.014496740000000008, False)
Slab: (C24Mn56, (1, 1, 0), 0.04730328499999997, False)
Slab: (C24Mn56, (1,

In [ ]:
from collections import Counter
from ocdata.core.bulk import Bulk

# Initialize a dictionary to store bulk IDs and source IDs for two-element bulks
two_element_bulk_ids = {}

# Iterate over the bulk database to find two-element bulks
for idx, bulk_entry in enumerate(bulk_db):
    bulk_atoms = bulk_entry['atoms']
    elements = bulk_atoms.get_chemical_symbols()
    element_count = Counter(elements)

    if len(element_count) == 2 and 'C' in elements:
        two_element_bulk_ids[idx] = bulk_entry.get('src_id', None)

# Now, two_element_bulk_ids contains the bulk_id and optionally src_id for each two-element bulk
print("Two-element bulk IDs and Source IDs:", two_element_bulk_ids)


Two-element bulk IDs and Source IDs: {1585: 'mp-570791', 1593: 'mp-9947', 1594: 'mp-21256', 1610: 'mp-569989', 1618: 'mp-2318', 1621: 'mp-570641', 1629: 'mp-1591', 1634: 'mp-1094034', 1636: 'mp-22749', 1639: 'mp-631', 1643: 'mp-570112', 1648: 'mp-567505', 1650: 'mp-2034', 1664: 'mp-684989', 1680: 'mp-11713', 1687: 'mp-1021328', 1700: 'mp-974437', 1702: 'mp-8062', 1707: 'mp-567397', 1709: 'mp-20648', 1715: 'mp-910', 1726: 'mp-27814', 1734: 'mp-1018050', 1738: 'mp-568656', 1741: 'mp-1008632', 1746: 'mp-2305', 1747: 'mp-27919', 1756: 'mp-582034', 1759: 'mp-19855', 1762: 'mp-7140', 1765: 'mp-568696', 1766: 'mp-684602', 1767: 'mp-1096986', 1779: 'mp-20937', 1789: 'mp-2760', 1792: 'mp-568619', 1795: 'mp-10721', 1804: 'mp-1552', 1813: 'mp-7631', 1821: 'mp-11714', 1824: 'mp-28930', 1825: 'mp-985782', 1832: 'mp-1894', 1840: 'mp-570690', 1847: 'mp-1282', 1849: 'mp-567551', 1852: 'mp-32679', 1867: 'mp-866659', 1868: 'mp-568735', 1875: 'mp-542730', 1878: 'mp-28731'}


KeyboardInterrupt: ignored

여기 아래부터는 관련없는건데, 참고할코드있으면 갖다쓰셈. mainly oc22_dataset모듈이용해서 짠거.


In [ ]:
from ocdata import vasp
import importlib
importlib.reload(vasp)


<module 'ocdata.vasp' from '/content/ocp/Open-Catalyst-Dataset/Open-Catalyst-Dataset/ocdata/vasp.py'>

In [ ]:
from ocdata.oc22_dataset.surface import Slab, SlabGenerator, generate_all_slabs
from pymatgen.entries.computed_entries import ComputedStructureEntry
import time
import random

json_file_path = "unary_list.json"
all_pbx_stable_entries = [ComputedStructureEntry.from_dict(d) for d in \
                          json.load(open(json_file_path))]

def generate_test_bulk(all_pbx_stable_entries):
    entry = random.choice(all_pbx_stable_entries)  # Use random.choice to select a random entry
    test_bulk = entry.structure
    print('Bulk:',test_bulk.formula)  # You can remove this line if you don't want to print the result
    return test_bulk

def generate_random_sa_metal(all_metal_name_list):
    entry = random.choice(all_metal_name_list)  # Use random.choice to select a random entry
    sa = Molecule([entry], [[0, 0, 0]])
    print('SA:',sa.formula)  # You can remove this line if you don't want to print the result
    return sa

In [ ]:
o_ref = Molecule(['O'], [[0, 0, 0]])
print(o_ref.species[0].atomic_radius)

0.6 ang


In [ ]:
from ocdata.oc22_dataset.MXide_adsorption import MXideAdsorbateGenerator
from ocdata.oc22_dataset.termination_generator import get_random_clean_slabs
from pymatgen.core.structure import Structure, Lattice, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from ocdata import vasp
import os
from pymatgen.core.periodic_table import ElementBase

out_directory = './output_500'
out_directory_slab = './output_500/slab'
out_directory_adslab = './output_500/adslab'
os.makedirs(out_directory, exist_ok=True)
os.makedirs(out_directory_adslab, exist_ok=True)
os.makedirs(out_directory_slab, exist_ok=True)

o_ref = Molecule(['O'], [[0, 0, 0]])
o_ref_radii = o_ref.species[0].atomic_radius
slab_list=[]
adslab_list =[]
def get_tol(bulk,SA):

  sa_radius = SA.species[0].atomic_radius
  elements = bulk.composition.elements
  metal = [element for element in elements if element.name != 'O']
  bulk_metal_radius = metal[0].atomic_radius
  min_m_m_dist=sa_radius + bulk_metal_radius
  min_m_o_dist=o_ref_radii+sa_radius
  tol=min_m_m_dist/min_m_o_dist
  print(tol)
  return tol

count = 0
total_suf_sa_num = 300 # total_suf_sa_num

while count < total_suf_sa_num:

  bulk = generate_test_bulk(all_pbx_stable_entries)
  SA = generate_random_sa_metal(all_metal_name_list)
  slab_start = time.time()
  random_slab=get_random_clean_slabs(bulk, 1, 1, 6, 15, lll_reduce=False, in_unit_planes=False, primitive=False,
                            max_normal_search=None, min_lw=8.0, xsite='O', height=0.9,
                            r=None, max_ssize=80, check_polar=True) # bulk, n_slabs, max_index, min_slab_size
  if not random_slab:
      print('NO VIABLE CLEAN SLABS')
      continue
  slab_end = time.time()
  print('Sampled %s clean slabs in %.3f' % (random_slab[0].formula, slab_end-slab_start))
  ads_start = time.time()
  # now get random adsorbed slabs
  rand_slab= random.choice(random_slab)
  mxide_adsgen = MXideAdsorbateGenerator(random_slab[0], height=1.25, tol=2, ads_dist_is_blength=False, ads_dist=4,)
  adslab=mxide_adsgen.generate_random_adsorption_structure(SA, 1, max_coverage=1)
  if not adslab:
      print('NO VIABLE adslab')
      continue
  #################### ASE format and fix atom ############
  ase_adslab=AseAtomsAdaptor.get_atoms(adslab[0])
  z_positions=[atom.position[2] for atom in ase_adslab]
  c = FixAtoms(indices=[atom.index for atom in ase_adslab if atom.position[2] <np.median(z_positions)])
  ase_adslab.set_constraint(c)
  #########################################################
  ase_slab=AseAtomsAdaptor.get_atoms(random_slab[0])
  ase_slab_z_positions=[atom.position[2] for atom in ase_slab]
  c = FixAtoms(indices=[atom.index for atom in ase_slab if atom.position[2] <np.median(ase_slab_z_positions)])
  ase_slab.set_constraint(c)
  #########################################################

  #visual(ase_adslab, './', (300, 300),2)
  subdirectory = f'{count}'
  out_directory_adslab
  adslab_subdir_path = os.path.join(out_directory_adslab, subdirectory)
  slab_subdir_path = os.path.join(out_directory_slab, subdirectory)
  os.makedirs(adslab_subdir_path, exist_ok=True)
  os.makedirs(slab_subdir_path, exist_ok=True)
  vasp.write_vasp_input_files(ase_adslab, outdir=adslab_subdir_path, vasp_flags=None)
  vasp.write_vasp_input_files(ase_slab, outdir=slab_subdir_path, vasp_flags=None)

  ads_end=time.time()

  print(f'saved{count} out of 300 in {slab_end-slab_start}')
  slab_list.append(ase_slab)
  adslab_list.append(ase_adslab)

  count= count + 1


### 이제 다운로드 받고, gemnet으로 돌려보기

In [ ]:
!zip -r output_300.zip output_500

In [ ]:
from ase.io import Trajectory, write
write('./output_ase/adslab_list.traj',adslab_list)
write('./output_ase/slab_list.traj',slab_list)

In [ ]:
!zip -r output_ase_before_relax.zip output_ase

  adding: output_ase/ (stored 0%)
  adding: output_ase/slab_list.traj (deflated 75%)
  adding: output_ase/adslab_list.traj (deflated 75%)


In [ ]:
gemnet = get_checkpoint('GemNet-OC OC20+OC22')
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
calc = OCPCalculator(checkpoint_path=os.path.expanduser(gemnet), cpu=True)

/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [ ]:
!cp -r output_300.zip /content/drive/MyDrive/sa_generation_1029

In [ ]:
!cp -r output_ase /content/drive/MyDrive/sa_generation_1029

In [ ]:
from ase.optimize import LBFGS
from ase.build import fcc111, molecule, add_adsorbate
from ase.constraints import ExpCellFilter, StrainFilter

def energy_cal(atoms, fmax=0.05):
    atoms.set_calculator(calc)
    tags = np.ones(len(atoms))
    atoms.set_tags(tags)
    print(tags)
    opt1 = LBFGS(atoms)
    opt1.run(fmax=fmax)
    return atoms.get_total_energy()
def status(proceed):
    num = int(proceed/5)
    return '🟩'*num+'⬜️'*(20-num)
def oc_calculator(data, fmax_ref, calculator):
    E_tot_list=[]
    for i, d in enumerate(data):
        print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<', i+1, ' out of ', len(data), '>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        E_tot = energy_cal(d, fmax=fmax_ref)
        E_tot_list.append(round(E_tot, 3))
        proceed = round((i+1) / len(data) * 100, 2)
        print(status(proceed), proceed, '% done')
    return E_tot_list

e_adslab_tot=oc_calculator(adslab_list,0.05,calc)
e_slab_tot=oc_calculator(slab_list,0.05,calc)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 1  out of  300 >>>>>>>>>>>>>>>>>>>>>>>>>>>>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


       Step     Time          Energy         fmax
LBFGS:    0 10:54:20     -332.383453        2.9147
LBFGS:    1 10:54:22     -332.986115        2.3258
LBFGS:    2 10:54:23     -333.911957        5.1333
LBFGS:    3 10:54:24     -334.466675        0.9354
LBFGS:    4 10:54:26     -334.632965        0.6657
LBFGS:    5 10:54:28     -334.735901        0.8263
LBFGS:    6 10:54:30     -334.732300        0.5414
LBFGS:    7 10:54:32     -334.798004        0.5122
LBFGS:    8 10:54:33     -334.850067        0.3800
LBFGS:    9 10:54:35     -334.923401        0.3248
LBFGS:   10 10:54:36     -334.971649        0.3983
LBFGS:   11 10:54:38     -335.069000        0.3648
LBFGS:   12 10:54:39     -335.118927        0.2740
LBFGS:   13 10:54:40     -335.172363        0.2795
LBFGS:   14 10:54:42     -335.235168        0.3010
LBFGS:   15 10:54:44     -335.284302        0.3595
LBFGS:   16 10:54:45     -335.307678        0.3503
LBFGS:   17 10:54:47     -335.287537        0.2767
LBFGS:   18 10:54:49     -335.30

KeyboardInterrupt: ignored



```
from ocdata.oc22_dataset.surface import Slab, SlabGenerator, generate_all_slabs
from pymatgen.entries.computed_entries import ComputedStructureEntry

import time
import random

json_file_path = "unary_list.json"
all_pbx_stable_entries = [ComputedStructureEntry.from_dict(d) for d in \
                          json.load(open(json_file_path))]


def generate_test_bulk(all_pbx_stable_entries, seed=0):
    random.seed(seed)
    tot_start = time.time()
    entry = random.sample(all_pbx_stable_entries, 1)[0]
    test_bulk = entry.structure
    print('Bulk:',test_bulk.formula)  # You can remove this line if you don't want to print the result
    return test_bulk
def generate_random_sa_metal(all_metal_name_list, seed=0):
    random.seed(seed)
    tot_start = time.time()
    entry = random.sample(all_metal_name_list, 1)[0]
    sa = Molecule([entry], [[0, 0, 0]])
    print('SA:',sa.formula)  # You can remove this line if you don't want to print the result
    return sa

from ocdata.oc22_dataset.MXide_adsorption import MXideAdsorbateGenerator
from ocdata.oc22_dataset.termination_generator import get_random_clean_slabs
from pymatgen.core.structure import Structure, Lattice, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms

bulk=generate_test_bulk(all_pbx_stable_entries)
SA = generate_random_sa_metal(all_metal_name_list)

random_slab=get_random_clean_slabs(bulk, 1, 2, 10, 15, lll_reduce=False, in_unit_planes=False, primitive=False,
                           max_normal_search=None, min_lw=8.0, xsite='O', height=0.9,
                           r=None, max_ssize=100, check_polar=True) # bulk, n_slabs, max_index, min_slab_size,
print()
mxide_adsgen = MXideAdsorbateGenerator(random_slab[0], height=1.25, repeat=[1,1,1], r=random)
adslab=mxide_adsgen.generate_random_adsorption_structure(SA, 1, max_coverage=1)

ase_adslab=AseAtomsAdaptor.get_atoms(adslab[0])
z_positions=[atom.position[2] for atom in ase_adslab]
c = FixAtoms(indices=[atom.index for atom in ase_adslab if atom.position[2] <np.median(z_positions)])
ase_adslab.set_constraint(c)
visual(ase_adslab, './', (300, 300),2)

from ocdata import vasp
import os
out_directory = './output_dir'
subdirectory = '1'
subdir_path = os.path.join(out_directory, subdirectory)
os.makedirs(subdir_path, exist_ok=True)
vasp.write_vasp_input_files(ase_adslab, outdir=subdir_path, vasp_flags=None)

```




이렇게도 만들 수 있음.
```
slab_list=generate_all_slabs(test_bulk,1,10,15) #structure, max_index, min_slab_size, min_vacuum_size,bonds=None, tol=0.1, ftol=0.1
print('===========================================')
print(len(slab_list))
indices=slab_list[0].get_surface_sites()

print(indices['top'])
random_site = random.choice(indices['top'])
# Print the random site
print(random_site)
last_element = random_site[-1]
print(last_element)

slab_list[0].add_adsorbate_atom([last_element],'Pt',0.9)
slab_list[0]
print(len(indices['top']))
print(len(slab_list))
a=slab_list[0].num_sites  # 이걸로 if statement만들기 cutoff max num atom < 100
print(a)
```

